In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Functional API
In this module, we will explore Keras's another model building API called functional API.
Here is the documentation: https://keras.io/getting-started/functional-api-guide/

In `Sequential`, you build a model by adding a layer to the `model` object:

```
model = Sequential()
model.add(Conv2D(32, 3, input_shape=(32,32,3), activation='relu'))
model.add(Conv2D(32, 3))
model.add(Activation('relu'))
```

However, in functional API, you define the inpute parameters and pass the input into the appropiate layer and the subsequent output is passed as input into the appropiate layer.  Lastly you define the inputs and outputs in a Model layer:   
```
inputs = Input(shape=(32,32,3))
conv1 = Conv2D(32,3, activation='relu')(inputs)
conv2 = Conv2D(32,3)(conv1)
conv2 = BatchNormalization()(conv2)
conv2 = Activation('relu')(conv2)
output = Dense(10, activation='softmax')(conv2)

model = Model(inputs=inputs, outputs=output)
```
As you can see from above, the functional API model needs a separate layer for input and output. Input() creates the the input tensor. Model() makes a model object from the output tensor. 

An advantage of functional API over Sequential is its flexibility. You can build a model architecture that has parallel paths or merging layers.

```
conv1 = Conv2D(32,3,activation='relu')(conv0)
conv2 = Conv2D(32,5,activation='relu')(conv0)
concat = Concatenate(axis=-1)([conv1, conv2])
conv3 = Conv2D(32,3,activation='relu')(concat)

```
The above example shows that the feature map from conv0 is split into two paths which are passed to conv1, and conv2 and then concatenated before passing to conv3:

```

         ____ conv1 (3x3)____
        /                    \   
conv0---                      (concatenate) -- conv3 (3x3)
        \____ conv2 (5x5)____/
        
```

# Below is an example of building a Functional API model using the CIFAR10 dataset

In [ ]:
from tensorflow import tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model # This will print model architecture.
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation # We add the Concatentate function
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, SeparableConv2D # We will use AveragePooling2D. Similar to MaxPooling but now we take the average value in the window.
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


# set np and tf random seed for reproducibility

# load data

# Reshape xblock data and normalize


# convert to labels to categorical


#split into validation and train data


#Here is an example on how to build a Functional API architecture

In [ ]:
backend.clear_session()

# Input model
visible = Input(shape=(32,32,3))

# Create Tower 1
conv11 = Conv2D(32, (3,3), padding = 'same', activation='relu')(visible)
pool11 = MaxPooling2D((2, 2), padding = 'same')(conv11)

# Create Tower 2
conv21 = Conv2D(32, (3,3), padding = 'same', activation='relu')(visible)
pool21 = MaxPooling2D((2,2), padding = 'same')(conv21)

# Concatentate
merge = Concatenate(axis=-1)([pool11, pool21])

# Flatten into fully connected layer
flat = Flatten()(merge)

# Hidden connected layer and output
hidden = Dense(32, activation='relu')(flat)
drop = Dropout(0.5)(hidden)
output = Dense(10, activation='softmax')(drop)

model_example = Model(inputs=visible, outputs=output)
# plot graph
plot_model(model_example)

# Your Turn.  Use the CIFAR10 dataset throughout the assignment

### Build the Inception model found in figure 7.8 in Deep Learning with Python (Chollet) 
### Print (plot) the model architecture

In [ ]:
# Build and print (plot) the model architecture found in figure 7.8 (Chollet)

### Build the model that is shown in the Week_5_API_Model_Architecture.png figure under the Week 5 Content tab.  Choose the number of hidden units (nodes) in each layer.  Then compile, fit, plot, and assess on the test data all in one cell. 

In [ ]:
# Build the model and print (plot) the model.

In [ ]:
#Compile, fit, plot, and assess in one cell.  

### Optimize your neural network. Try using SeparableConv2D layers (Chollet 7.3.1) add or remove layers, change the number of units (nodes), add or remove regularization and normalization. Spend sometime optimizing the network using the techniques we have learned thus far.  Feel free to do your own research on best arcitectures.  Can you improve your test accuracy above 0.80%, which was the benchmark last week? Remember to compile, plot, assess on test data all in one cell.

In [ ]:
# Optimize your model(s).
# Build the model(s) and print (plot) the model.

In [ ]:
# Compile, fit, plot, and assess in one cell.

### What are the benifits of Funtional API models.  Give an hypothetical example (different than the book) where having different input types would be benificial in API models.  Give another hypotheical exmple (different than the book) where having different output types would be benificial in API models

### Comment on the model arcitecture that worked the best. What was the accuracy? Did you improve accuracry from last week?  How long did the training take?